In [1]:
from metavision_core.event_io import EventsIterator, load_events
import tonic
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn as nn
import torch
import models
import random
from IPython.utils import io
# import sdl2.ext


In [2]:
# %matplotlib inline
%matplotlib ipympl
# %matplotlib tk
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["image.cmap"] = "bone"

In [3]:

file_name = "1hzplane"

# ievs = load_events("./data/Line/" + file_name + ".dat")
# ievs = np.load("./data/Line/" + file_name + ".npy")
# ifs = torch.from_numpy(
#     np.load("./data/Line/"+file_name+"_frames.npy")[0:2000,:,:]
#     )
# if torch.cuda.is_available():
#   ifs = ifs.cuda(0)



In [4]:
i_evs = tonic.datasets.NMNIST(save_to="./data") # (34, 34, 2) 3
# i_evs = tonic.datasets.POKERDVS(save_to="./data") # (35, 35, 2) 3
# i_evs = tonic.datasets.DVSGesture(save_to="./data") # (128, 128, 2) 2
# i_evs = tonic.datasets.ASLDVS(save_to="./data") # (240, 180, 2) 3

i_evs.sensor_size

(34, 34, 2)

In [17]:
to_frame = tonic.transforms.ToFrame(i_evs.sensor_size, n_time_bins=10)

events, label = i_evs[random.randint(0, len(i_evs)-1)]
events = events.squeeze()
events = np.array(list(filter(lambda ev: ev[3]==0, events)))
# print(events)
frames = to_frame(events)[3:8]
with io.capture_output() as captured:
    ani = tonic.utils.plot_animation(frames)
print(label)
ani

9


In [18]:
ani.save("./img/nmnist-9.gif", writer="imagemagick")

In [ ]:
def n_evs(evs):
    sum = 0
    for i, ev in enumerate(evs):
        # print(i)
        sum = sum + (ev[0].size-sum)/(i+1)
    print(sum)
n_evs(i_evs)
# MNIST: 4172
# POKERDVS: 2991
# DVSGesture: 361903
# ASLDVS: 28149
# CIFAR10DVS: 204409


In [ ]:
i_evs[0][0][-1]

In [ ]:
def t_evs(evs):
    sum = 0
    for i, ev in enumerate(evs):
        # print(i)
        sum = sum + (ev[0][-1][0]-sum)/(i+1)
    print(sum)

t_evs(i_evs)
# MNIST: 307663
# POKERDVS: 17283
# DVSGesture: 6455240
# ASLDVS: 110247
# CIFAR10DVS: 1293438


In [ ]:


to_frame = tonic.transforms.ToFrame(
    sensor_size=i_evs.sensor_size,
    time_window=361
)
noise = tonic.transforms.UniformNoise(
    sensor_size=i_evs.sensor_size,
    n=n_evs(i_evs)*0.1,
)


In [ ]:

animation = tonic.utils.plot_animation(frames=ifs)
animation = tonic.utils.plot_animation(frames=ofs)


In [ ]:

net = models.SNN2()


In [ ]:

with torch.inference_mode():
    ofs = net(ifs)

# np.save("./data/Line/" + file_name + "_out.npy")


In [ ]:

animation = tonic.utils.plot_animation(frames=ofs.cpu())
